In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from math import ceil
from scipy.linalg import sqrtm
from scipy.sparse.linalg import svds
from scipy.sparse import lil_matrix, csr_matrix, diags

## Algorithm

In [ ]:
def gen_columns(M):
    '''
    Yileds columns of matrix M. Prevents from out-of-order access
    to matrix columns. It can be used to simulate data streaming.
    '''
    for i in range(M.shape[1]):
        yield M[:, i]

In [ ]:
def sample_entry(M, delta):
    
    A = lil_matrix(M.shape, dtype=np.float64)
    
    for i in range(M.shape[1]):
        sample = np.random.choice(2, M.shape[0], p=[1 - delta, delta])
        A[:, i] = (M[:, i] * sample).reshape(-1, 1)

    return A

In [ ]:
def SPCA(C, k):
    omega = np.random.randn(C.shape[1], k)
    omega = csr_matrix(omega)
    
    _C = lil_matrix(C.shape)
    
    # Trimming
    for i in range (C.shape[0]):
        if (C.getrow(i).count_nonzero() > 10):
            _C[i, :] = 0
        else:
            _C[i, :] = C[i, :]
    
    _C = _C.tocsr()
    
    diag_matrix = diags(diagonals=((_C.T @ _C).diagonal()), offsets=0, format="csr")
    
    F = _C.T @ _C - diag_matrix
    F = F.todense()
    power = int(np.ceil(5 * np.log(C.shape[1])))
        
    F = np.linalg.matrix_power(F, power)
    F = F @ omega
    Q, _ = np.linalg.qr(F)

    return csr_matrix(Q[:, :k])

In [ ]:
def SLA(M_cols, m, n, k, delta, l):
    
   #l_samples = list(np.random.choice(n, l, replace=True))
    l_samples = np.arange(l)
    
    M_l = np.empty((m, l)) # M_l is matrix of first l columns
    
    for j in l_samples:
        M_l[:, j] = next(M_cols)
        
    A_b1 = sample_entry(M_l, delta).tocsr()
    A_b2 = sample_entry(M_l, delta)
    
    del M_l
    
    Q = SPCA(A_b1, k)
    
    for i in range(m):
        if ((A_b2.getrow(i).count_nonzero()) > 2):
            A_b2[i, :] = 0
            
    for i in range(l):
        if ((A_b2.getcol(i).count_nonzero()) > 10 * delta * m):
            A_b2[:, i] = 0
    
    A_b2 = A_b2.tocsr()
    W = A_b2 @ Q
    V = lil_matrix((n, k))
    
    V[:l, :] = A_b1.T @ W
    I = A_b1 @ V[:l, :]
    
    for t in range(l, n):
        sample = np.random.choice(2, m, p=[1 - delta, delta])
        A_t = csr_matrix(next(M_cols) * sample)
        V[t, :] = A_t.reshape(1, -1) @ W
        I += A_t.reshape(-1, 1) @ V[t, :].reshape(1, -1)
    
    V = V.tocsr()
    R = sqrtm((V.T @ V).todense())
    R = np.linalg.inv(R)
    
    U = (1.0 / delta) * (I @ R @ R.T)    
    M_k = U @ V.T
    
    return M_k      

## Evaluation

In [ ]:
import scipy.sparse.linalg as lin

In [ ]:
def compress(image, k): 
    if len(image.shape) == 2:
        image = np.reshape(image, newshape=(image.shape[0],image.shape[1],1))
    reconst_matrix = np.empty(shape=image.shape)
    for channel in range(image.shape[2]):
        u,s,v = lin.svds(image[:,:,channel],k)
        reconst_matrix[:,:,channel] = u@np.diag(s)@v
    reconst_matrix = np.clip(reconst_matrix, 0, 255).astype(np.uint8)
    return reconst_matrix

In [ ]:
def genenerate_time_matrices_k(**kwargs):
    import time
    import copy
    
    k_array = np.array(kwargs['k_array'])
    delta = kwargs['delta']
    l = kwargs['l']
    n_repeats = kwargs['n_repeats']    
    
    sla_compressor = lambda x,k,m,n: SLA(x, delta=delta, k=k, m=m, n=n, l=l)
    svd_compressor = lambda x,k: compress(x,k)
        
    powers = [3,3.1,3.2,3.3,3.4,3.5,3.6]#,3.7,3.8,4.]
    svd_time = np.empty(shape=(len(powers), len(k_array)))
    sla_time = np.empty(shape=(len(powers), len(k_array)))
    
    np.random.seed(30)
    for i,p in enumerate(powers):            
            svd_round_time = np.empty(len(k_array))
            sla_round_time = np.empty(len(k_array))
            
            #for _ in range(n_repeats):
            for j,k in enumerate(k_array):
                img = np.random.rand(int(10**p),int(10**p))

                #for j,k in enumerate(k_array):
                for _ in range(n_repeats):
                    img_col = gen_columns(copy.copy(img))
                    m,n = img.shape
                    tpoint = time.time()
                    sla_compressor(img_col,k,m,n)
                    sla_round_time[j] = time.time()-tpoint
                    
                    tpoint = time.time()
                    svd_compressor(img,k)
                    svd_round_time[j] = time.time()-tpoint

            svd_time[i,:] = svd_round_time/n_repeats
            sla_time[i,:] = sla_round_time/n_repeats
            
    return powers, k_array, svd_time, sla_time

In [ ]:
def compare_plot(powers, k_array, svd_time, sla_time):
    plt.figure(figsize=(12,8))
    for i in range(len(k_array)):
        plt.plot(powers, svd_time[:,i], label='svd, k={}'.format(k_array[i]))
        plt.plot(powers, sla_time[:,i], label='sla, k={}'.format(k_array[i]))
    #plt.plot(powers, svd_time_d1, label='ssss')
    plt.yscale('log')
    plt.legend(fontsize=18)
    plt.title("Complexity plot in loglog scale", fontsize=18)
    plt.ylabel('Time',fontsize=18)
    plt.xlabel('Matrix size',fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.show()

In [ ]:
powers, k_array, svd_time, sla_time = genenerate_time_matrices(k_array=[8,10,12],l=20,delta=.1,n_repeats=10)

In [ ]:
compare_plot(powers[0:7], k_array, svd_time[0:7,:], sla_time[0:7,:])

In [ ]:
def genenerate_time_matrices_d(**kwargs):
    import time
    import itertools
    
    #k_array = kwargs['k_array']
    k = kwargs['k_array']
    delta = np.array(kwargs['delta'])
    l = kwargs['l']
    n_repeats = kwargs['n_repeats']    
    
    sla_compressor = lambda x,delta,m,n: SLA(x, delta=delta, k=k, m=m, n=n, l=l)
    svd_compressor = lambda x: compress(x,k)
    
    #params = 
        
    powers = [3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,4.]
    svd_time = np.empty(len(powers))
    sla_time = np.empty(shape=(len(powers), len(delta)))
    
    np.random.seed(30)
    for i,p in enumerate(powers):            
        svd_round_time = []
        sla_round_time = np.empty(len(delta))
            
        #for _ in range(n_repeats):
        for j,d in enumerate(delta):
            img = np.random.rand(int(10**p),int(10**p))

            #for j,k in enumerate(k_array):
            for _ in range(n_repeats):
                img_col = gen_columns(img)
                m,n = img.shape
                tpoint = time.time()
                sla_compressor(img_col,d,m,n)
                sla_round_time[j] = time.time()-tpoint
                
                if j == 0:
                    tpoint = time.time()
                    svd_compressor(img)
                    svd_round_time.append(time.time()-tpoint)

        svd_time[i] = np.mean(svd_round_time)
        sla_time[i,:] = sla_round_time/n_repeats
            
    return powers, delta, svd_time, sla_time

In [ ]:
powers_d, k_array_d, svd_time_d, sla_time_d = genenerate_time_matrices(k_array=10,l=20,\
                                                                       delta=[.1,.2,.4,.07],n_repeats=10)

In [ ]:
def compare_plot(powers, k_array, svd_time, sla_time):
    plt.figure(figsize=(12,8))
    for i in range(len(k_array)):
        plt.plot(powers, sla_time[:,i], label='sla, k={}'.format(k_array[i]))
        
    plt.plot(powers, svd_time, label='svd')
    plt.yscale('log')
    plt.legend(fontsize=18)
    plt.title("Complexity plot in loglog scale", fontsize=18)
    plt.ylabel('Time',fontsize=18)
    plt.xlabel('Matrix size',fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.show()

In [ ]:
compare_plot(powers_d, k_array_d, svd_time_d, sla_time_d)

In [ ]:
def compare_by_delta(matrix_size, k, l, n_repeats=30 ):
    import time
    import copy
    
    sla_compressor = lambda x,m,n, delta: SLA(x, delta=delta, k=k, m=m, n=n, l=l)
    
    delta_array = np.arange(start=.01,stop=.4,step=.03)
    
    times = []
    img = np.random.rand(int(10**matrix_size),int(10**matrix_size))
    m,n = img.shape
    for delta in delta_array:
        print(delta)
        round_time = []
        for _ in range(n_repeats):
            img_col = gen_columns(copy.copy(img))
            tpoint = time.time()
            sla_compressor(img_col,m,n, delta)
            round_time.append(time.time()-tpoint)
        times.append(np.mean(round_time))
    
    return delta_array, times

In [ ]:
def compare_by_k(matrix_size, delta, l, n_repeats=10 ):
    import time
    import copy
    
    sla_compressor = lambda x,m,n,k: SLA(x, delta=delta, k=k, m=m, n=n, l=l)
    
    k_array = np.arange(1,np.int(l/2), step=2)
    
    times = []
    img = np.random.rand(int(10**matrix_size),int(10**matrix_size))
    m,n = img.shape
    for k in k_array:
        print(k)
        round_time = []
        for _ in range(n_repeats):
            img_col = gen_columns(copy.copy(img))
            tpoint = time.time()
            sla_compressor(img_col,m,n,k)
            round_time.append(time.time()-tpoint)
        times.append(np.mean(round_time))
    
    return k_array, times

In [ ]:
k_array, k_times = compare_by_k(3.4, .1, 20)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(k_array, k_times)

In [ ]:
delta_array1, delta_times1 = compare_by_delta(3.4, 10, 20, 10)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(delta_array1, delta_times1)
plt.xlabel(r'$\delta$', fontsize=15)
plt.ylabel('Time', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.title(r'Time($\delta$) for matrix from $\mathbb{R}^{2500\times 2500}$', fontsize=15)
plt.show()